In [ ]:
from google.colab import drive
drive.mount('/content/drive')
%cd '/content/drive/MyDrive/EEG project'

In [ ]:
# !pip install mne
import numpy as np
import os
import mne
from mne.io import concatenate_raws, read_raw_edf

In [ ]:
def get_paths(path,task_num,participant_num):
  list_a = [] #存放所有.edf文件的路径
  for i in range(1,participant_num+1):
    if i < 10:
      i = '0'+str(i)
    else:
      i = str(i)
    for j in task_num:
      a = os.path.join(path, "S0"+i, "S0"+i+'R'+j+'.edf')
      list_a.append(a)
  return list_a

def get_data(p_list):
  X,y = [],[]
  for i in p_list:
    raw=read_raw_edf(i,preload=False)
    events_from_annot, event_dict = mne.events_from_annotations(raw)
    raw.pick_channels(['Cz..','C3..','C4..'])
    raw.resample(sfreq=100)
    epochs = mne.Epochs(raw, events=events_from_annot, event_id=event_dict, tmin=-2, tmax=2,
                      baseline=None, preload=True)
    labels = list(event_dict.keys())
    for l in labels:
        for epoch in epochs[l]:
          e = np.delete(epoch,-1,axis=1)  #把641变成640(401变400)
          e = e.T
          j = e.reshape((32,20,3))
          X.append(j)
          y.append(event_dict[l]-1)#..
  y = np.array(y)
  X = np.array(X)
  return (X,y)

dir_str = './MIdataset/files'
task_num = ['04','08','12']
participant_num = 85
list_a = get_paths(path=dir_str, task_num=task_num, participant_num=participant_num)
train_X,y = get_data(list_a)

np.save(file="train_eeg.npy", arr=train_X)
np.save(file="labels.npy", arr=y)
def normalization(data):
  _range = np.max(data) - np.min(data)
  return (data - np.min(data)) / _range
def preprocess(data):
  ampl_data = data*100000
  norm_data = normalization(ampl_data)
  rgb_data = norm_data*255
  rgb_data = rgb_data.astype(np.uint8)
  return rgb_data

rgb_data = preprocess(train_X)
np.save(file="rgb_eeg.npy", arr=rgb_data)
